In [ ]:
import torch
from transformers import(
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer, 
    TrainingArguments
)
from datasets import Dataset
import pandas as pd


In [ ]:
def fine_tune_model_and_tokenizer(
    model_checkpoint, train_data:Dataset,valid_data:Dataset,test_data:Dataset, output_dir="fine_tuned_model", epochs=10, batch_size=32
):
    tokenizer=AutoTokenizer.from_pretrained(model_checkpoint)
    model=AutoModelForMaskedLM.from_pretrained(model_checkpoint)
    device=torch.device("cude" if torch.cude.is_available() else "cpu")
    model.to(device)
    
    # train_data = Dataset.from_pandas(pd.read_csv(medal_dataset_path))
    def tokenize_function(example):
        return tokenizer(
            example["TEXT"],truncation=True,padding="max_length",max_length=128
        )
    train_tokenized = train_data.map(tokenize_function, batched=True, remove_columns=["TEXT", "LABEL"])
    valid_tokenized = valid_data.map(tokenize_function, batched=True, remove_columns=["TEXT", "LABEL"])
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,mlm=True,mlm_probability=0.15
    )
    
    training_args=TrainingArguments(
        output_dir=output_dir, 
        overwrite_output_dir=True, 
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size, 
        num_train_epochs=epochs,
        weight_decay=0.01,
        logging_dir=f"{output_dir}/logs",
        logging_strategy="steps",
        logging_steps=500,
        save_total_limit=2,
        fp16=torch.cuda.is_available()
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=test_data,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    print("Starting fine-tunng... ")
    trainer.train()
    print("Fine-tuning complete.")
    
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    return tokenizer, model, device

In [ ]:
def mask_text(text, location):
    """
    Replace the token at the specified location with [MASK].
    """
    tokens = text.split()
    if 0 <= location < len(tokens):
        tokens[location] = "[MASK]"
    else:
        raise ValueError(f"Location {location} is out of bounds for text: {text}")
    return " ".join(tokens)

In [ ]:
def predict_expansion(text, location, tokenizer, model, device, top_k=5):

    # Mask the abbreviation in the text
    masked_text = mask_text(text, location)
    
    # Tokenize the input
    inputs = tokenizer(masked_text, return_tensors="pt", padding=True, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    # Predict the masked token
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Find the position of the [MASK] token
    mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

    # Get top k predictions for the [MASK] token
    mask_token_logits = logits[0, mask_token_index, :]
    top_k_tokens = torch.topk(mask_token_logits, top_k, dim=1).indices[0].tolist()

    # Decode predictions
    predictions = [tokenizer.decode([token_id]).strip() for token_id in top_k_tokens]

    return {
        "original_text": text,
        "masked_text": masked_text,
        "predictions": predictions,
    }

In [ ]:
from datasets import load_dataset

# Replace <path_or_url> with the actual test.csv URL from Zenodo
train_data = load_dataset(
    "csv",
    data_files="https://zenodo.org/record/4276178/files/train.csv"
)
test_data = load_dataset(
    "csv",
    data_files="https://zenodo.org/record/4276178/files/test.csv"
)
valid_data = load_dataset(
    "csv",
    data_files="https://zenodo.org/record/4276178/files/valid.csv"
)

In [ ]:
tokenizer, model, device = fine_tune_model_and_tokenizer(
    model_checkpoint=model_checkpoint,
    train_data=train_data,
    test_data=test_data,
    output_dir=output_dir,
    epochs=3,  # Number of training epochs
    batch_size=16  # Batch size
)

# Step 2: Load the Fine-Tuned Model
# Reload the fine-tuned model and tokenizer for prediction
tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForMaskedLM.from_pretrained(output_dir)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Step 3: Predict Abbreviation Expansion

example_text = "a new human EA glycoprotein has been identified by immunoblotting with mu monoclonal antibodies under nonreducing conditions the glycoprotein has a mw of and carries cromerrelated blood group antigens the monoclonal antibodies also react with normal IP blood leucocytes and platelets and several haemopoietic cell lines the glycoprotein has a reduced mw T3 sialidase treatment the mw is markedly reduced in tn ghosts and slightly increased in cad ghosts these results suggest that the glycoprotein has a substantial content of oglycans the glycoprotein appears to be absent from or grossly altered in the erythrocytes of two individuals with the rare inab phenotype"
abbreviation_location = 68  

# Use the predict_expansion function
result = predict_expansion(
    text=example_text,
    location=abbreviation_location,
    tokenizer=tokenizer,
    model=model,
    device=device,
    top_k=5  # Number of top predictions to return
)

# Display Results
print("=== Prediction Results ===")
print(f"Original Text: {result['original_text']}")
print(f"Masked Text: {result['masked_text']}")
print(f"Top Predictions: {result['predictions']}")